In [2]:
import pandas as pd
import os    

In [3]:
# On définit les colonnes du Dataset
column_names = ["name", "cel", "cla", "flu", "gac", "gel", "org", "pia", "sax", "tru", "vio", "voi"]

df = pd.DataFrame(columns=column_names)
dataset_folder = "./IRMAS-TestingData"

# Pour chaque fichier txt du dataset on crée une ligne dans le dataframe avec le nom du fichier et les instruments présents
for file_name in os.listdir(dataset_folder):
    if file_name.endswith(".txt"):
        file_path = os.path.join(dataset_folder, file_name)
        with open(file_path, "r") as f:
            instrument_abbreviations = f.read().split("\t\n")

        row = [file_name[:-4]] + [1 if abbreviation.strip() in instrument_abbreviations else 0 for abbreviation in column_names[1:]]
        # -4 pour enlever le .txt
        
        df.loc[len(df)] = row

# Save DataFrame as a CSV file
df.to_csv("dataset.csv", index=False)